# Lab B.4: Evaluation and Analysis

**Module:** B - Recommender Systems  
**Time:** 1.5 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Implement and understand ranking metrics (NDCG, MAP, MRR)
- [ ] Compare all recommender models on a unified benchmark
- [ ] Analyze cold start performance
- [ ] Evaluate recommendation diversity
- [ ] Understand production deployment considerations

---

## 📚 Prerequisites

- Completed: Labs B.1, B.2, B.3
- Trained models from previous notebooks

---

## 🌍 Real-World Context

**The Metric Matters:** Optimizing the wrong metric can destroy your product.

- 📺 **Netflix**: Optimized for "thumbs up" → users gave good ratings but didn't watch
- 🎵 **Spotify**: Learned that "completion rate" beats "skip rate" for user satisfaction
- 🛒 **Amazon**: "Purchase" matters more than "click"

This notebook teaches you to measure what matters!

---

## 🧒 ELI5: Ranking Metrics

> **Imagine you're a teacher grading search results...**
>
> **Hit Rate (HR):** "Did ANY good answer appear in the top 10?" (Pass/Fail)
>
> **NDCG:** "Did the BEST answers appear at the TOP?" (Position matters!)
> - Good answer at #1: Full credit
> - Good answer at #10: Partial credit
> - Good answer at #100: Almost no credit
>
> **MRR:** "Where's the FIRST good answer?" (Only the first one counts)
> - First good at #1: Score = 1.0
> - First good at #2: Score = 0.5
> - First good at #10: Score = 0.1
>
> **MAP:** "On average, how good is the precision at each relevant position?"

---

## Part 1: Setup and Model Training

In [ ]:
import sys
from pathlib import Path

module_dir = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(module_dir / 'scripts'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from data_utils import download_movielens, leave_one_out_split, compute_statistics
from metrics import (
    hit_rate_at_k, ndcg_at_k, mrr, recall_at_k, 
    mean_average_precision, evaluate_ranking, format_metrics
)

np.random.seed(42)
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Using device: {device}")

In [ ]:
# Load data
ratings_df, movies_df = download_movielens('100k')

# Leave-one-out split
train_df, test_df = leave_one_out_split(ratings_df, by_time=True)

num_users = ratings_df['user_id'].nunique()
num_items = ratings_df['item_id'].nunique()

# Build user -> positive items mapping
user_positive_items = train_df.groupby('user_id')['item_id'].apply(set).to_dict()

print(f"📊 Dataset: {num_users} users, {num_items} items")
print(f"   Train: {len(train_df):,}, Test: {len(test_df):,}")

### Train All Models

Let's train our models from previous notebooks for fair comparison.

In [ ]:
# Import model classes
from models import MatrixFactorization, NeuMF

# Prepare data for implicit feedback
class ImplicitDataset(Dataset):
    def __init__(self, interactions_df, num_items, user_positive_items, num_negatives=4):
        self.users = interactions_df['user_id'].values
        self.items = interactions_df['item_id'].values
        self.num_items = num_items
        self.user_positive_items = user_positive_items
        self.num_negatives = num_negatives
        
    def __len__(self):
        return len(self.users)
    
    def __getitem__(self, idx):
        user = self.users[idx]
        pos_item = self.items[idx]
        
        neg_items = []
        positives = self.user_positive_items.get(user, set())
        
        while len(neg_items) < self.num_negatives:
            neg_item = np.random.randint(0, self.num_items)
            if neg_item not in positives:
                neg_items.append(neg_item)
        
        users = [user] * (1 + self.num_negatives)
        items = [pos_item] + neg_items
        labels = [1.0] + [0.0] * self.num_negatives
        
        return (
            torch.LongTensor(users),
            torch.LongTensor(items),
            torch.FloatTensor(labels)
        )

def collate_fn(batch):
    users = torch.cat([b[0] for b in batch])
    items = torch.cat([b[1] for b in batch])
    labels = torch.cat([b[2] for b in batch])
    return users, items, labels


train_dataset = ImplicitDataset(train_df, num_items, user_positive_items, num_negatives=4)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, collate_fn=collate_fn)

In [ ]:
def train_model(model, train_loader, device, epochs=10, lr=0.001):
    """Train a model for specified epochs."""
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        
        for users, items, labels in train_loader:
            users, items, labels = users.to(device), items.to(device), labels.to(device)
            
            predictions = model(users, items)
            
            # Handle different model outputs
            if predictions.dim() == 0:
                predictions = predictions.unsqueeze(0)
            
            # Sigmoid for models that don't have it
            if predictions.max() > 1 or predictions.min() < 0:
                predictions = torch.sigmoid(predictions)
            
            loss = F.binary_cross_entropy(predictions, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
    
    return model


# Train Matrix Factorization
print("Training Matrix Factorization...")
mf_model = MatrixFactorization(num_users, num_items, embedding_dim=64)
mf_model = train_model(mf_model, train_loader, device, epochs=10)
print("✅ MF trained")

# Train NeuMF
print("\nTraining NeuMF...")
neumf_model = NeuMF(num_users, num_items, gmf_dim=32, mlp_dim=64)
neumf_model = train_model(neumf_model, train_loader, device, epochs=10)
print("✅ NeuMF trained")

---

## Part 2: Implementing Evaluation Metrics

In [ ]:
def evaluate_model(model, test_df, user_positive_items, device, k_values=[5, 10, 20]):
    """
    Comprehensive evaluation of a recommender model.
    
    For each user in test set:
    1. Predict scores for all items
    2. Mask out training items
    3. Compute ranking metrics
    """
    model.eval()
    model.to(device)
    
    all_ranked_items = []
    all_ground_truths = []
    
    test_users = test_df['user_id'].unique()
    
    with torch.no_grad():
        for user_id in tqdm(test_users, desc='Evaluating'):
            # Get ground truth
            test_item = test_df[test_df['user_id'] == user_id]['item_id'].values[0]
            
            # Predict all items
            user_ids = torch.LongTensor([user_id] * num_items).to(device)
            item_ids = torch.arange(num_items).to(device)
            
            scores = model(user_ids, item_ids)
            if scores.dim() == 0:
                scores = scores.unsqueeze(0)
            
            # Mask training items
            train_items = list(user_positive_items.get(user_id, set()))
            scores[train_items] = -float('inf')
            
            # Get ranked items
            _, ranked_items = torch.sort(scores, descending=True)
            ranked_items = ranked_items.cpu().numpy()
            
            all_ranked_items.append(ranked_items[:max(k_values)])
            all_ground_truths.append([test_item])
    
    # Compute metrics
    results = {}
    
    for k in k_values:
        # Hit Rate
        hr_scores = [hit_rate_at_k(r, g, k) for r, g in zip(all_ranked_items, all_ground_truths)]
        results[f'HR@{k}'] = np.mean(hr_scores)
        
        # NDCG
        ndcg_scores = [ndcg_at_k(r, g, k) for r, g in zip(all_ranked_items, all_ground_truths)]
        results[f'NDCG@{k}'] = np.mean(ndcg_scores)
        
        # Recall (same as HR for single test item)
        results[f'Recall@{k}'] = results[f'HR@{k}']
    
    # MRR
    mrr_scores = [mrr(r, g) for r, g in zip(all_ranked_items, all_ground_truths)]
    results['MRR'] = np.mean(mrr_scores)
    
    return results, all_ranked_items


# Evaluate models
print("\n" + "="*60)
print("EVALUATING MODELS")
print("="*60)

mf_results, mf_rankings = evaluate_model(mf_model, test_df, user_positive_items, device)
print("\n📊 Matrix Factorization Results:")
for metric, value in sorted(mf_results.items()):
    print(f"   {metric}: {value:.4f}")

neumf_results, neumf_rankings = evaluate_model(neumf_model, test_df, user_positive_items, device)
print("\n📊 NeuMF Results:")
for metric, value in sorted(neumf_results.items()):
    print(f"   {metric}: {value:.4f}")

---

## Part 3: Visual Comparison

In [ ]:
# Prepare comparison data
models = ['Matrix Factorization', 'NeuMF']
all_results = [mf_results, neumf_results]

# Create comparison DataFrame
comparison_data = []
for model_name, results in zip(models, all_results):
    for metric, value in results.items():
        comparison_data.append({
            'Model': model_name,
            'Metric': metric,
            'Value': value
        })

comparison_df = pd.DataFrame(comparison_data)

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# HR@K comparison
hr_metrics = ['HR@5', 'HR@10', 'HR@20']
x = np.arange(len(hr_metrics))
width = 0.35

mf_hr = [mf_results[m] for m in hr_metrics]
neumf_hr = [neumf_results[m] for m in hr_metrics]

axes[0].bar(x - width/2, mf_hr, width, label='Matrix Factorization', color='#3498db')
axes[0].bar(x + width/2, neumf_hr, width, label='NeuMF', color='#e74c3c')
axes[0].set_xlabel('Metric')
axes[0].set_ylabel('Score')
axes[0].set_title('Hit Rate Comparison')
axes[0].set_xticks(x)
axes[0].set_xticklabels(hr_metrics)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# NDCG@K comparison
ndcg_metrics = ['NDCG@5', 'NDCG@10', 'NDCG@20']
mf_ndcg = [mf_results[m] for m in ndcg_metrics]
neumf_ndcg = [neumf_results[m] for m in ndcg_metrics]

axes[1].bar(x - width/2, mf_ndcg, width, label='Matrix Factorization', color='#3498db')
axes[1].bar(x + width/2, neumf_ndcg, width, label='NeuMF', color='#e74c3c')
axes[1].set_xlabel('Metric')
axes[1].set_ylabel('Score')
axes[1].set_title('NDCG Comparison')
axes[1].set_xticks(x)
axes[1].set_xticklabels(ndcg_metrics)
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

# MRR comparison
mrr_values = [mf_results['MRR'], neumf_results['MRR']]
colors = ['#3498db', '#e74c3c']
axes[2].bar(models, mrr_values, color=colors, edgecolor='black')
axes[2].set_ylabel('MRR Score')
axes[2].set_title('Mean Reciprocal Rank Comparison')
axes[2].grid(axis='y', alpha=0.3)

# Add value labels
for i, v in enumerate(mrr_values):
    axes[2].text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

---

## Part 4: Cold Start Analysis

How do our models perform for users with few ratings?

In [ ]:
# Categorize users by activity level
user_activity = train_df.groupby('user_id').size()

# Define activity buckets
def get_activity_bucket(num_ratings):
    if num_ratings <= 20:
        return 'Cold (≤20)'
    elif num_ratings <= 50:
        return 'Warm (21-50)'
    elif num_ratings <= 100:
        return 'Active (51-100)'
    else:
        return 'Power (>100)'

user_buckets = user_activity.apply(get_activity_bucket)

# Show distribution
print("📊 User Activity Distribution:")
print(user_buckets.value_counts().sort_index())

In [ ]:
def evaluate_by_bucket(model, test_df, user_positive_items, user_buckets, device, k=10):
    """Evaluate model performance by user activity bucket."""
    model.eval()
    model.to(device)
    
    bucket_results = defaultdict(list)
    
    with torch.no_grad():
        for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc='Evaluating'):
            user_id = row['user_id']
            test_item = row['item_id']
            bucket = user_buckets.get(user_id, 'Unknown')
            
            # Predict
            user_ids = torch.LongTensor([user_id] * num_items).to(device)
            item_ids = torch.arange(num_items).to(device)
            scores = model(user_ids, item_ids)
            
            # Mask training items
            train_items = list(user_positive_items.get(user_id, set()))
            scores[train_items] = -float('inf')
            
            # Get ranking
            _, ranked_items = torch.sort(scores, descending=True)
            ranked_items = ranked_items.cpu().numpy()[:k]
            
            # Calculate metrics
            hr = hit_rate_at_k(ranked_items, [test_item], k)
            bucket_results[bucket].append(hr)
    
    # Aggregate
    return {bucket: np.mean(hrs) for bucket, hrs in bucket_results.items()}


# Evaluate both models by bucket
print("\nEvaluating Matrix Factorization by user activity...")
mf_bucket_results = evaluate_by_bucket(mf_model, test_df, user_positive_items, user_buckets, device)

print("Evaluating NeuMF by user activity...")
neumf_bucket_results = evaluate_by_bucket(neumf_model, test_df, user_positive_items, user_buckets, device)

In [ ]:
# Plot cold start analysis
buckets = ['Cold (≤20)', 'Warm (21-50)', 'Active (51-100)', 'Power (>100)']
buckets = [b for b in buckets if b in mf_bucket_results]  # Only include existing buckets

mf_hrs = [mf_bucket_results.get(b, 0) for b in buckets]
neumf_hrs = [neumf_bucket_results.get(b, 0) for b in buckets]

x = np.arange(len(buckets))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, mf_hrs, width, label='Matrix Factorization', color='#3498db')
bars2 = ax.bar(x + width/2, neumf_hrs, width, label='NeuMF', color='#e74c3c')

ax.set_xlabel('User Activity Level')
ax.set_ylabel('Hit Rate @ 10')
ax.set_title('Cold Start Analysis: HR@10 by User Activity')
ax.set_xticks(x)
ax.set_xticklabels(buckets)
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height, f'{height:.3f}',
            ha='center', va='bottom', fontsize=9)
for bar in bars2:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height, f'{height:.3f}',
            ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\n📊 Observations:")
print("   - Cold start users (≤20 ratings) have lower hit rates")
print("   - Neural models can better leverage sparse signals")
print("   - Power users benefit most from collaborative signals")

---

## Part 5: Diversity Analysis

Are we recommending a diverse set of items, or just popular ones?

In [ ]:
def analyze_diversity(rankings, movies_df, num_items, top_k=10):
    """
    Analyze the diversity of recommendations.
    
    Metrics:
    - Coverage: What fraction of items are ever recommended?
    - Genre diversity: How many different genres in recommendations?
    - Popularity bias: Are we only recommending popular items?
    """
    # Coverage
    all_recommended = set()
    for ranking in rankings:
        all_recommended.update(ranking[:top_k])
    coverage = len(all_recommended) / num_items
    
    # Genre diversity per user
    genre_diversities = []
    for ranking in rankings:
        genres = set()
        for item_id in ranking[:top_k]:
            movie_genres = movies_df[movies_df['item_id'] == item_id]['genres'].values
            if len(movie_genres) > 0 and pd.notna(movie_genres[0]):
                genres.update(movie_genres[0].split('|'))
        genre_diversities.append(len(genres))
    
    # Recommendation frequency
    item_counts = defaultdict(int)
    for ranking in rankings:
        for item_id in ranking[:top_k]:
            item_counts[item_id] += 1
    
    return {
        'coverage': coverage,
        'avg_genre_diversity': np.mean(genre_diversities),
        'item_counts': dict(item_counts),
        'unique_items_recommended': len(all_recommended)
    }


# Analyze diversity
mf_diversity = analyze_diversity(mf_rankings, movies_df, num_items)
neumf_diversity = analyze_diversity(neumf_rankings, movies_df, num_items)

print("📊 Diversity Analysis:")
print("\nMatrix Factorization:")
print(f"   Coverage: {mf_diversity['coverage']:.2%} ({mf_diversity['unique_items_recommended']} items)")
print(f"   Avg genres per recommendation: {mf_diversity['avg_genre_diversity']:.1f}")

print("\nNeuMF:")
print(f"   Coverage: {neumf_diversity['coverage']:.2%} ({neumf_diversity['unique_items_recommended']} items)")
print(f"   Avg genres per recommendation: {neumf_diversity['avg_genre_diversity']:.1f}")

In [ ]:
# Visualize popularity bias
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# MF popularity distribution
mf_counts = sorted(mf_diversity['item_counts'].values(), reverse=True)
axes[0].bar(range(len(mf_counts[:100])), mf_counts[:100], alpha=0.7)
axes[0].set_xlabel('Item Rank (by recommendation frequency)')
axes[0].set_ylabel('Times Recommended')
axes[0].set_title('Matrix Factorization: Recommendation Frequency')
axes[0].grid(axis='y', alpha=0.3)

# NeuMF popularity distribution
neumf_counts = sorted(neumf_diversity['item_counts'].values(), reverse=True)
axes[1].bar(range(len(neumf_counts[:100])), neumf_counts[:100], alpha=0.7, color='orange')
axes[1].set_xlabel('Item Rank (by recommendation frequency)')
axes[1].set_ylabel('Times Recommended')
axes[1].set_title('NeuMF: Recommendation Frequency')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Observation: Long tail shows some items are recommended much more often")
print("   This is typical popularity bias - popular items get recommended more.")

---

## Part 6: Production Recommendations

Based on our analysis, here are recommendations for production deployment:

In [ ]:
# Create comprehensive summary table
summary_data = {
    'Metric': ['HR@10', 'NDCG@10', 'MRR', 'Coverage', 'Avg Genres/Rec'],
    'Matrix Factorization': [
        f"{mf_results['HR@10']:.4f}",
        f"{mf_results['NDCG@10']:.4f}",
        f"{mf_results['MRR']:.4f}",
        f"{mf_diversity['coverage']:.2%}",
        f"{mf_diversity['avg_genre_diversity']:.1f}"
    ],
    'NeuMF': [
        f"{neumf_results['HR@10']:.4f}",
        f"{neumf_results['NDCG@10']:.4f}",
        f"{neumf_results['MRR']:.4f}",
        f"{neumf_diversity['coverage']:.2%}",
        f"{neumf_diversity['avg_genre_diversity']:.1f}"
    ]
}

summary_df = pd.DataFrame(summary_data)

print("\n" + "="*60)
print("FINAL MODEL COMPARISON")
print("="*60)
print(summary_df.to_string(index=False))
print("="*60)

In [ ]:
# Production recommendations
print("""
╔══════════════════════════════════════════════════════════════╗
║          PRODUCTION DEPLOYMENT RECOMMENDATIONS               ║
╠══════════════════════════════════════════════════════════════╣
║                                                              ║
║  1. MODEL SELECTION                                          ║
║     ├─ Start with Matrix Factorization (simpler, faster)     ║
║     ├─ Graduate to NeuMF for better accuracy                 ║
║     └─ Use Two-Tower for candidate retrieval at scale        ║
║                                                              ║
║  2. COLD START HANDLING                                      ║
║     ├─ New users: Use content-based until enough data        ║
║     ├─ New items: Feature-based embeddings (Two-Tower)       ║
║     └─ Fallback: Popular/trending items                      ║
║                                                              ║
║  3. DIVERSITY                                                ║
║     ├─ Add MMR (Maximal Marginal Relevance) re-ranking       ║
║     ├─ Inject exploration (10-20% random/trending)           ║
║     └─ Monitor coverage metrics in production                ║
║                                                              ║
║  4. EVALUATION                                               ║
║     ├─ A/B test with engagement metrics (CTR, watch time)    ║
║     ├─ Monitor for feedback loops and filter bubbles         ║
║     └─ Balance accuracy vs diversity vs novelty              ║
║                                                              ║
║  5. DGX SPARK ADVANTAGES                                     ║
║     ├─ 128GB memory: Full user/item embeddings in GPU        ║
║     ├─ FAISS GPU: Sub-millisecond retrieval                  ║
║     └─ Large batches: Better in-batch negative sampling      ║
║                                                              ║
╚══════════════════════════════════════════════════════════════╝
""")

---

## ✋ Try It Yourself!

### Exercise 1: Implement NDCG with Graded Relevance

Our current NDCG uses binary relevance. Implement NDCG with graded relevance based on actual ratings.

<details>
<summary>💡 Hint</summary>

Instead of relevance = 1 for any rated item, use:
- relevance = (rating - 1) / 4  for 1-5 ratings → 0 to 1
- Or use the rating directly: relevance = rating

</details>

In [ ]:
# YOUR CODE HERE



### Exercise 2: Implement MMR Re-ranking

Implement Maximal Marginal Relevance to balance relevance and diversity:

$$MMR = \arg\max_{d \in R\setminus S} \left[ \lambda \cdot Rel(d, q) - (1-\lambda) \cdot \max_{d' \in S} Sim(d, d') \right]$$

<details>
<summary>💡 Hint</summary>

1. Start with top item by relevance
2. For each next item, trade off relevance vs similarity to already-selected items
3. λ=1 is pure relevance, λ=0 is pure diversity

</details>

In [ ]:
# YOUR CODE HERE



---

## ⚠️ Common Mistakes

### Mistake 1: Evaluating on Training Items

In [ ]:
# ❌ Wrong: Include training items in evaluation
# rankings = model.get_top_k(user)  # Might include items user already rated
# hr = compute_hr(rankings, test_item)  # Artificially high!

# ✅ Right: Mask training items
# scores[train_items] = -inf
# rankings = get_top_k(scores)

print("Why this matters:")
print("  Without masking: HR@10 could be 95%+ (trivially high)")
print("  With masking: Measures true discovery ability")

### Mistake 2: Confusing Metrics

In [ ]:
# Common confusion: Hit Rate vs Recall

print("Hit Rate vs Recall:")
print("")
print("HR@K:  'Was ANY relevant item in top K?'")
print("       → Binary: 0 or 1")
print("       → For 1 test item: HR@K = Recall@K")
print("")
print("Recall@K: 'What fraction of relevant items are in top K?'")
print("          → Range: 0 to 1")
print("          → If user has 5 relevant items and 3 are in top K: Recall = 0.6")
print("")
print("For leave-one-out (1 test item): HR = Recall")
print("For multiple test items: Recall is more informative")

---

## 🎉 Module Complete!

Congratulations! You've completed the Recommender Systems module. You've learned:

- ✅ **Lab B.1**: Matrix Factorization and collaborative filtering fundamentals
- ✅ **Lab B.2**: Neural Collaborative Filtering (GMF + MLP = NeuMF)
- ✅ **Lab B.3**: Two-Tower retrieval with FAISS for production scale
- ✅ **Lab B.4**: Comprehensive evaluation with ranking metrics

### Key Takeaways

1. **Start simple**: Matrix factorization is a strong baseline
2. **Scale smartly**: Two-tower + FAISS for billion-scale retrieval
3. **Measure right**: NDCG > accuracy for ranking tasks
4. **Balance trade-offs**: Accuracy vs diversity vs novelty
5. **DGX Spark advantage**: 128GB memory enables large-scale experimentation

---

## 📖 Further Reading

- [Recommender Systems Handbook](https://www.springer.com/gp/book/9780387858203) - Comprehensive reference
- [Microsoft Recommenders](https://github.com/microsoft/recommenders) - Production-quality implementations
- [RecBole](https://recbole.io/) - Unified recommendation framework
- [Deep Learning for Recommender Systems](https://dl.acm.org/doi/10.1145/3285029) - Academic survey

---

## 🧹 Cleanup

In [ ]:
import gc

del mf_model, neumf_model
torch.cuda.empty_cache()
gc.collect()

print("✅ GPU memory cleared!")
print("\n🎉 Module B: Recommender Systems - Complete!")